In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Import useful libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# <font color='red'>**Loading trained networks**</font>

In [ ]:
path_model = "../models/clasification/Vgg16HumedaFinalV2.h5"
model = tf.keras.models.load_model(path_model, compile=False)
model.summary()

In [ ]:
l1 = model.get_layer(name='global_average_pooling2d')
emb = Model(model.input, l1.output)
emb.summary()

# Getting the data

In [ ]:
#loading the wet data
group = "Humeda"

if group == "Humeda":
    df = pd.read_csv("../data/rgb/classification/augmented/Humeda.csv")
else:
    df = pd.read_csv("../data/rgb/classification/augmented/Seca.csv")
    
df.columns = ['path', 'label']

#making the train and test splits
train_df, test_df = train_test_split(df, test_size=0.2, random_state=14)

print(train_df.groupby('label').count())
print(test_df.groupby('label').count())

In [ ]:
cuni1_df = test_df[test_df['label'] == 'CuNi1']
cuni2_df = test_df[test_df['label'] == 'CuNi2']
cuni3_df = test_df[test_df['label'] == 'CuNi3']

cuni1_features, cuni2_features, cuni3_features = [], [], []

#getting the features for Cuni1
for i in range (len(cuni1_df)):
    img = tf.keras.preprocessing.image.load_img(cuni1_df.iloc[i]['path'], target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    features = emb.predict(img_array)
    cuni1_features.extend(features)

#getting the features for Cuni2
for i in range (len(cuni2_df)):
    img = tf.keras.preprocessing.image.load_img(cuni2_df.iloc[i]['path'], target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    features = emb.predict(img_array)
    cuni2_features.extend(features)

#getting the features for Cuni3
for i in range (len(cuni3_df)):
    img = tf.keras.preprocessing.image.load_img(cuni3_df.iloc[i]['path'], target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    features = emb.predict(img_array)
    cuni3_features.extend(features)

#saving the features
np.save("../features/"+group+"/CuNi1", cuni1_features)
np.save("../features/"+group+"/CuNi2", cuni2_features)
np.save("../features/"+group+"/CuNi3", cuni3_features)


In [ ]:
#loading the np files
group = "Humeda"

cuni1_features = np.load("../features/"+group+"/CuNi1.npy")
cuni2_features = np.load("../features/"+group+"/CuNi2.npy")
cuni3_features = np.load("../features/"+group+"/CuNi3.npy")

print("dimension features for wet set:")
print(cuni1_features.shape)
print(cuni2_features.shape)
print(cuni3_features.shape)

In [ ]:
features_mix = np.concatenate((cuni1_features, cuni2_features, cuni3_features), axis=0)
features_mix.shape

# Features decomposition in eigenvalues and eigenvectors

In [ ]:
mat_cov = np.cov(features_mix)
mat_cov

In [ ]:
eig_vals, eig_vectors = np.linalg.eig(mat_cov)
print("Valores propios \n%s" %eig_vals)
print("Vectores propios \n%s" %eig_vectors)

In [ ]:
mat_corr = np.corrcoef(features_mix)
mat_corr.shape